In [10]:
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, CData, Tag
from io import BytesIO
import PyPDF2
import ssl
import certifi

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys


class MyBeautifulSoup(BeautifulSoup):
    def _all_strings_plus(  self, strip=True, types=NavigableString, 
                                aRef={'a': lambda a: f"<{a.get('href', '')}>"}, 
                                skipTags=['script', 'style']    ):
        # verify types
        if hasattr(types,'__iter__') and not isinstance(types,type):
            types = tuple([t for t in types if isinstance(t, type)])
        if not (types and isinstance(types,(type,tuple))): types = NavigableString
        
        # skip text in tags included in aRef
        skipTags += list(aRef.keys())
        
        for descendant in self.descendants:
            # yield extra strings according to aRef
            if isinstance(descendant, Tag) and descendant.name in aRef:
                extraStr = aRef[descendant.name](descendant)
                if isinstance(extraStr, str): yield extraStr

            # skip text nodes DIRECTLY inside a Tag in aRef
            if descendant.parent.name in aRef: continue

            # skip ALL text nodes inside skipTags 
            if skipTags and descendant.find_parent(skipTags): continue

            # default behavior
            if not isinstance(descendant, types): continue

            if strip:
                descendant = descendant.strip()
                if len(descendant) == 0: continue
            yield descendant
    
    def get_text_plus(self, separator=" ", srcUrl=None, **aspArgs):
        if srcUrl and isinstance(srcUrl, str):
            def hrefStr(aTag):
                href = aTag.get('href')
                if not (href is None or href.startswith('javascript')):
                    return f"<{urljoin(srcUrl, href)}>"
            aspArgs.setdefault('aRef', {})
            aspArgs['aRef']['a'] = hrefStr
        
        return separator.join(self._all_strings_plus(**aspArgs)) 

In [5]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

### BeautifulSoup & PDF testing

In [ ]:
source_html = requests.get('https://azure-na-assets.contentstack.com/v3/assets/blt71bfe6e8a1c2d265/bltc320e2642a070852/Exelon_Environment_Policy_Poster-8.5x11v2.pdf')

In [ ]:
raw_data = source_html.content
with BytesIO(raw_data) as data:
    read_pdf = PyPDF2.PdfFileReader(data)

    for page in range(read_pdf.getNumPages()):
        print(read_pdf.getPage(page).extractText())

In [32]:
session = requests.Session()
url = 'https://data.sec.gov/submissions/CIK0000009466.json'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'host': 'www.sec.gov'}
session.headers.update(headers)
response = session.get(url, verify=certifi.where())


In [33]:
response

<Response [404]>

In [ ]:
soup = MyBeautifulSoup(source_html.text, 'html.parser')
# soup.text

In [ ]:
for script in soup(['script', 'style']):
    script.decompose()
len(soup.get_text_plus())

In [ ]:
text_links_parsed = ' '.join(soup.get_text_plus().split())

In [ ]:
soup.get_text_plus()

### Selenium Testing

In [104]:
# Function to recursively find text in all iframes in a web page. Maybe I dont need this. 
def find_all_iframes(driver) -> list:
    iframes = driver.find_elements_by_xpath("//iframe")
    text_per_frame = []
    print(iframes)
    frame_soup = MyBeautifulSoup(driver.page_source, 'html.parser')
    frame_text_links = frame_soup.get_text_plus()
    text_per_frame.append(frame_text_links)
    for index, iframe in enumerate(iframes):
        # Your sweet business logic applied to iframe goes here.
        driver.switch_to.frame(index)
        # html = driver.execute_script("return document.body.innerHTML;")
        frame_soup = MyBeautifulSoup(driver.page_source, 'html.parser')
        frame_text_links = frame_soup.get_text_plus()
        text_per_frame.append(frame_text_links)
        nested_iframe_text = find_all_iframes(driver)
        if nested_iframe_text:
            text_per_frame.append(nested_iframe_text)
        driver.switch_to.parent_frame()
        
    return text_per_frame


def url_inline_text(soup):
    # Remove script and style elements
    for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()
    
    # # Remove comment elements
    # for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
    #     comment.extract()
    
    # Replace links with text and URL
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('javascript') or len(a['href']) < 2:
            a.decompose()
        else:
            a.replace_with(f"{a.get_text()} ({a['href']})")

    return soup

def get_shadow_root_text(driver, wait) -> list:
    # Augment HTML with text in shadow DOM
    shdw_dom_text = []
    all_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*')))
    for el in all_elements:
        try:
            if driver.execute_script('return arguments[0].shadowRoot', el):
                
                shdw_el = driver.execute_script(f"return document.querySelector('{el.tag_name}').shadowRoot.querySelector('div')")
                # print(shdw_el.find_elements(By.CSS_SELECTOR, "p"))
                frame_soup = url_inline_text(BeautifulSoup(shdw_el.get_attribute('innerHTML'), 'html.parser'))
                # frame_text_links = frame_soup.get_text_plus()
                # print('-'*25)
                # print(frame_soup.get_text(separator='\n'))
                shdw_dom_text.append(frame_soup.get_text(separator=' | '))
                
        except Exception as e:
            pass

    return shdw_dom_text

In [156]:
# Attribute that contains HTML of current page. Reference to Selenium methods
drop_downs = driver.find_element(By.CSS_SELECTOR, "body")#.shadow_root
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
# spans = driver.find_elements_by_class_name('euds-accordion')
# print([span.text for span in spans])

# shadow_dict = driver.execute_script('return arguments[0].shadowRoot', drop_downs)
data = driver.execute_script('return arguments[0].shadowRoot', drop_downs)
data

In [39]:
# Create web driver object
from selenium.webdriver.common.by import By
driver = webdriver.Chrome('C:\\Users\\14102\\Brown\\Internships\\INL\\CRMM_Scraping\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe')

In [75]:
# Method to navigate to provided URL
actions = ActionChains(driver)
wait = WebDriverWait(driver, 20)
driver.get('https://www.bge.com/safety-community/environment/our-initiatives')

In [114]:
# Get HTML from main DOM
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup = url_inline_text(soup)
# Get text, excluding any residual JavaScript or CSS
main_html_text = soup.get_text(separator=' | ', strip=True)

# Get any text from shadow root
shadow_html_text_list = get_shadow_root_text(driver, wait)

full_html = '|'.join([main_html_text] + shadow_html_text_list)

In [116]:
driver.quit()

In [111]:
shadow_html_text_list

[]